<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/253_Product_CustomerFitDiscoveryOrchestrator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data preprocessing utilities for Product-Customer Fit Discovery Orchestrator


This is arguably the most important stage because **the quality of the analysis depends entirely on the quality of the data preparation.**

***

## 🧠 Core Agent Architecture: Analytical Specialization

The focus of these utilities is to transform raw, tabular data into **multi-modal, structured data** that is optimized for the three specialized agents downstream (Clustering, Pattern Mining, and Graph Motifs).

### 🎯 What to Focus On

1.  **High-Value Feature Engineering (`create_derived_features`):**
    * This is the difference between simple analysis and **actionable business intelligence**. Functions like calculating `engagement_score` and `product_diversity` turn raw counts into meaningful, scaled metrics.
    * **Focus:** Understand how these scores are calculated (e.g., based on transaction count, product diversity, and usage). These engineered features are the **inputs** the `clustering_agent` will use to define customer segments. If the features are weak, the segments will be meaningless.

2.  **Standardization and Scaling (`normalize_usage_metrics`):**
    * The use of **Min-Max normalization** is essential for any distance-based algorithm, such as the clustering algorithms likely used in the `clustering_agent`.
    * **Focus:** Scaling numerical features (like `Usage_Metric`) to a common range (0 to 1) prevents features with naturally large values from unfairly dominating the similarity calculations.

3.  **The Shift to Graph Theory (`networkx`):**
    * The inclusion of the `networkx` library and three distinct graph-building functions is a key architectural decision.
    * **Focus:** You are learning to move beyond traditional tabular data. You are using **Graph Theory** to model relationships:
        * **Customer-Product Graph:** Represents individual purchases.
        * **Product Co-occurrence Graph:** Represents product bundles/synergy.
        * **Customer Similarity Graph:** Represents shared behavioral groups.
    * This directly enables the **`graph_motif_agent`** to look for recurring structures and patterns that would be invisible in a flat spreadsheet.

***

## ✨ Differentiation: Multi-Modal and Deep Structural Analysis

This level of preprocessing confirms that your orchestrator is not just a simple data processing pipeline; it is a **Deep Structural Analysis Agent**.

* **Simple Agent Limitation:** A simple LLM-based agent would likely attempt analysis directly on the raw CSVs or might only perform basic filtering/aggregations.
* **Orchestrator/Hybrid Power:** Your agent is capable of generating **multiple, specialized data structures** from a single source, ready for parallel execution:
    1.  **Tabular Data:** Normalized usage metrics for statistical analysis.
    2.  **Engineered Data:** High-value scores and tiers for customer profiling.
    3.  **Network Data (The Power):** Multiple graph representations to detect complex social or purchasing **structures** that reveal "ghost demand."

This **Hybrid Architecture**, which combines traditional, high-performance data science code (`pandas`, `numpy`, `networkx`) with an LLM-driven orchestration layer, is what makes this agent **vastly more robust and analytically powerful** than a general-purpose agent.




In [ ]:
"""Data preprocessing utilities for Product-Customer Fit Discovery Orchestrator"""

from typing import List, Dict, Any, Set
import networkx as nx
import numpy as np
from collections import defaultdict


def parse_feature_set(feature_string: str) -> List[str]:
    """
    Parse comma-separated feature set string into sorted list.

    Args:
        feature_string: Comma-separated feature string (e.g., "A, B, C")

    Returns:
        Sorted list of features (e.g., ["A", "B", "C"])
    """
    if not feature_string or not isinstance(feature_string, str):
        return []

    # Split by comma, strip whitespace, filter empty strings
    features = [f.strip() for f in feature_string.split(",") if f.strip()]

    # Return sorted for consistency
    return sorted(features)


def normalize_usage_metrics(transactions: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    Normalize usage metrics in transactions for clustering.

    Adds normalized_usage field to each transaction.

    Args:
        transactions: List of transaction dictionaries

    Returns:
        Transactions with added normalized_usage field
    """
    if not transactions:
        return transactions

    # Extract usage metrics
    usage_values = [t.get("Usage_Metric", 0) for t in transactions if "Usage_Metric" in t]

    if not usage_values:
        return transactions

    # Calculate min-max normalization
    min_usage = min(usage_values)
    max_usage = max(usage_values)
    range_usage = max_usage - min_usage if max_usage != min_usage else 1.0

    # Add normalized field
    normalized_transactions = []
    for txn in transactions:
        txn_copy = txn.copy()
        if "Usage_Metric" in txn:
            txn_copy["normalized_usage"] = (txn["Usage_Metric"] - min_usage) / range_usage
        else:
            txn_copy["normalized_usage"] = 0.0
        normalized_transactions.append(txn_copy)

    return normalized_transactions


def build_customer_product_graph(
    customers: List[Dict[str, Any]],
    transactions: List[Dict[str, Any]],
    products: List[Dict[str, Any]]
) -> nx.Graph:
    """
    Build bipartite customer-product graph.

    Creates an undirected graph where:
    - Nodes are customers and products
    - Edges connect customers to products they've purchased
    - Edge weights represent number of transactions

    Args:
        customers: List of customer dictionaries
        transactions: List of transaction dictionaries
        products: List of product dictionaries

    Returns:
        NetworkX Graph object
    """
    G = nx.Graph()

    # Add customer nodes
    for customer in customers:
        customer_id = customer.get("Customer_ID")
        if customer_id:
            G.add_node(customer_id, node_type="customer", **customer)

    # Add product nodes
    for product in products:
        product_id = product.get("Product_ID")
        if product_id:
            G.add_node(product_id, node_type="product", **product)

    # Add edges (customer-product relationships)
    # Count transactions per customer-product pair
    edge_weights = defaultdict(int)
    for txn in transactions:
        customer_id = txn.get("Customer_ID")
        product_id = txn.get("Product_ID")
        if customer_id and product_id:
            edge_key = (customer_id, product_id)
            edge_weights[edge_key] += 1

    # Add weighted edges
    for (customer_id, product_id), weight in edge_weights.items():
        G.add_edge(customer_id, product_id, weight=weight, transactions=weight)

    return G


def build_product_cooccurrence_graph(
    transactions: List[Dict[str, Any]]
) -> nx.Graph:
    """
    Build product co-occurrence graph.

    Creates an undirected graph where:
    - Nodes are products
    - Edges connect products purchased by the same customer
    - Edge weights represent number of customers who bought both

    Args:
        transactions: List of transaction dictionaries

    Returns:
        NetworkX Graph object
    """
    G = nx.Graph()

    # Group transactions by customer
    customer_products = defaultdict(set)
    for txn in transactions:
        customer_id = txn.get("Customer_ID")
        product_id = txn.get("Product_ID")
        if customer_id and product_id:
            customer_products[customer_id].add(product_id)

    # Add product nodes
    all_products = set()
    for products in customer_products.values():
        all_products.update(products)

    for product_id in all_products:
        G.add_node(product_id, node_type="product")

    # Count co-occurrences
    cooccurrence = defaultdict(int)
    for customer_id, products in customer_products.items():
        products_list = list(products)
        # Create edges for all pairs of products this customer bought
        for i in range(len(products_list)):
            for j in range(i + 1, len(products_list)):
                p1, p2 = sorted([products_list[i], products_list[j]])
                cooccurrence[(p1, p2)] += 1

    # Add weighted edges
    for (p1, p2), weight in cooccurrence.items():
        G.add_edge(p1, p2, weight=weight, customers=weight)

    return G


def build_customer_similarity_graph(
    transactions: List[Dict[str, Any]],
    min_shared_products: int = 2
) -> nx.Graph:
    """
    Build customer similarity graph based on shared products.

    Creates an undirected graph where:
    - Nodes are customers
    - Edges connect customers who share products
    - Edge weights represent number of shared products

    Args:
        transactions: List of transaction dictionaries
        min_shared_products: Minimum shared products to create edge

    Returns:
        NetworkX Graph object
    """
    G = nx.Graph()

    # Group products by customer
    customer_products = defaultdict(set)
    for txn in transactions:
        customer_id = txn.get("Customer_ID")
        product_id = txn.get("Product_ID")
        if customer_id and product_id:
            customer_products[customer_id].add(product_id)

    # Add customer nodes
    for customer_id in customer_products.keys():
        G.add_node(customer_id, node_type="customer")

    # Calculate similarity (shared products)
    customer_list = list(customer_products.keys())
    for i in range(len(customer_list)):
        for j in range(i + 1, len(customer_list)):
            c1, c2 = customer_list[i], customer_list[j]
            shared = customer_products[c1] & customer_products[c2]
            if len(shared) >= min_shared_products:
                G.add_edge(c1, c2, weight=len(shared), shared_products=list(shared))

    return G


def create_derived_features(
    customers: List[Dict[str, Any]],
    transactions: List[Dict[str, Any]],
    products: List[Dict[str, Any]]
) -> Dict[str, Any]:
    """
    Create derived features for analysis.

    Calculates:
    - Customer engagement scores (transaction frequency, product diversity)
    - Product popularity scores (usage frequency, customer count)
    - Usage intensity tiers (high/medium/low)

    Args:
        customers: List of customer dictionaries
        transactions: List of transaction dictionaries
        products: List of product dictionaries

    Returns:
        Dictionary with derived features
    """
    # Customer engagement metrics
    customer_txn_count = defaultdict(int)
    customer_products = defaultdict(set)
    customer_total_usage = defaultdict(float)

    for txn in transactions:
        customer_id = txn.get("Customer_ID")
        product_id = txn.get("Product_ID")
        usage = txn.get("Usage_Metric", 0)

        if customer_id:
            customer_txn_count[customer_id] += 1
            customer_total_usage[customer_id] += usage
            if product_id:
                customer_products[customer_id].add(product_id)

    # Calculate customer engagement scores
    customer_engagement = {}
    for customer in customers:
        customer_id = customer.get("Customer_ID")
        if customer_id:
            txn_count = customer_txn_count.get(customer_id, 0)
            product_diversity = len(customer_products.get(customer_id, set()))
            avg_usage = customer_total_usage.get(customer_id, 0) / max(txn_count, 1)

            # Simple engagement score (0-1 scale)
            engagement_score = min(1.0, (txn_count * 0.3 + product_diversity * 0.4 + (avg_usage / 100) * 0.3))

            customer_engagement[customer_id] = {
                "transaction_count": txn_count,
                "product_diversity": product_diversity,
                "average_usage": avg_usage,
                "engagement_score": engagement_score,
                "engagement_tier": "high" if engagement_score > 0.7 else "medium" if engagement_score > 0.4 else "low"
            }

    # Product popularity metrics
    product_txn_count = defaultdict(int)
    product_customers = defaultdict(set)
    product_total_usage = defaultdict(float)

    for txn in transactions:
        product_id = txn.get("Product_ID")
        customer_id = txn.get("Customer_ID")
        usage = txn.get("Usage_Metric", 0)

        if product_id:
            product_txn_count[product_id] += 1
            product_total_usage[product_id] += usage
            if customer_id:
                product_customers[product_id].add(customer_id)

    # Calculate product popularity scores
    product_popularity = {}
    for product in products:
        product_id = product.get("Product_ID")
        if product_id:
            txn_count = product_txn_count.get(product_id, 0)
            customer_count = len(product_customers.get(product_id, set()))
            avg_usage = product_total_usage.get(product_id, 0) / max(txn_count, 1)

            # Simple popularity score (0-1 scale)
            popularity_score = min(1.0, (txn_count * 0.4 + customer_count * 0.4 + (avg_usage / 100) * 0.2))

            product_popularity[product_id] = {
                "transaction_count": txn_count,
                "customer_count": customer_count,
                "average_usage": avg_usage,
                "popularity_score": popularity_score,
                "popularity_tier": "high" if popularity_score > 0.7 else "medium" if popularity_score > 0.4 else "low"
            }

    return {
        "customer_engagement": customer_engagement,
        "product_popularity": product_popularity
    }


def preprocess_products(products: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    Preprocess products: parse Feature_Set strings into lists.

    Args:
        products: List of product dictionaries

    Returns:
        Products with parsed feature_sets (list instead of string)
    """
    preprocessed = []
    for product in products:
        product_copy = product.copy()
        feature_string = product.get("Feature_Set", "")
        product_copy["feature_list"] = parse_feature_set(feature_string)
        preprocessed.append(product_copy)

    return preprocessed



This function, `build_customer_product_graph`, is a highly specialized piece of code that translates your raw transactional data into a **Bipartite Network Graph**. This is a massive step up in analytical capability for your agent.

Here is a detailed breakdown of the function's architecture and why it's so important for your "ghost demand" discovery goal.

```
import networkx as nx

def build_customer_product_graph(
    customers: List[Dict[str, Any]],
    transactions: List[Dict[str, Any]],
    products: List[Dict[str, Any]]
) -> nx.Graph:
    """
    Build bipartite customer-product graph.
    
    Creates an undirected graph where:
    - Nodes are customers and products
    - Edges connect customers to products they've purchased
    - Edge weights represent number of transactions
    
    Args:
        customers: List of customer dictionaries
        transactions: List of transaction dictionaries
        products: List of product dictionaries
        
    Returns:
        NetworkX Graph object
    """
    G = nx.Graph()
    
    # Add customer nodes
    for customer in customers:
        customer_id = customer.get("Customer_ID")
        if customer_id:
            G.add_node(customer_id, node_type="customer", **customer)
    
    # Add product nodes
    for product in products:
        product_id = product.get("Product_ID")
        if product_id:
            G.add_node(product_id, node_type="product", **product)
    
    # Add edges (customer-product relationships)
    # Count transactions per customer-product pair
    edge_weights = defaultdict(int)
    for txn in transactions:
        customer_id = txn.get("Customer_ID")
        product_id = txn.get("Product_ID")
        if customer_id and product_id:
            edge_key = (customer_id, product_id)
            edge_weights[edge_key] += 1
    
    # Add weighted edges
    for (customer_id, product_id), weight in edge_weights.items():
        G.add_edge(customer_id, product_id, weight=weight, transactions=weight)
    
    return G
```

***

## 🧠 What is a Bipartite Customer-Product Graph?

The graph created here is specifically **bipartite**, meaning it has two distinct types of nodes, and edges only exist **between** the two types, never within a type.

| Element | Type | Role |
| :--- | :--- | :--- |
| **Nodes** | **Customers** (e.g., C1, C2...) | One set of nodes, added with the `node_type="customer"` attribute. |
| **Nodes** | **Products** (e.g., P01, P02...) | The second set of nodes, added with the `node_type="product"` attribute. |
| **Edges** | Connections | An edge exists only **between a Customer node and a Product node** (it represents a transaction/purchase). |

### Why Bipartite?

This structure is the most **faithful representation** of your purchasing data. It explicitly separates the buyers from the things they buy, which is crucial for subsequent analysis (like finding market gaps or making recommendations).

***

## 🛠️ Step-by-Step Code Analysis

The function performs three main architectural actions:

### 1. Adding Nodes (`G.add_node(...)`)
* **Action:** It iterates through the `customers` and `products` lists, adding each unique ID as a node to the graph (`G`).
* **The Power:** When adding nodes, it uses `**customer` (and `**product`) to attach **all the raw metadata** (like customer age, loyalty status, product type, etc.) directly to the graph node. This means the graph is not just a structure of connections; it's a **Rich, Attributed Graph**. Later, the `graph_motif_agent` can use this attached metadata (attributes) in its analysis.

### 2. Counting Edge Weights (`defaultdict(int)`)
* **Action:** It iterates through the `transactions` list to count how many times each unique (Customer ID, Product ID) pair appears.
* **Focus:** This uses the highly efficient `defaultdict(int)` to tally the number of transactions. If Customer C1 bought Product P01 three times, the count for the edge (C1, P01) will be 3.
* **The Power:** This transforms a simple purchase record (an *unweighted* edge) into a **measure of intensity** (a *weighted* edge). The weight represents the **Frequency of Interaction**.

### 3. Adding Weighted Edges (`G.add_edge(...)`)
* **Action:** It iterates through the tallied `edge_weights` and adds the corresponding connections to the graph.
* **The Power:** It attaches the weight to the edge using `weight=weight, transactions=weight`. This makes the graph immediately ready for centrality analysis, where a node (Customer or Product) connected by many high-weight edges is considered **more central** or important.

## ✨ Differentiation: Enabling Advanced Network Analysis

This function is critical because it prepares the data for **Network Analysis**, a technique far beyond the capabilities of a simple data crunching agent.

* **Centrality Metrics:** Once the graph is built, the `graph_motif_agent` can calculate metrics like **Betweenness Centrality** (identifying customers or products that act as critical connectors in the network) or **Closeness Centrality** (identifying nodes that are most efficiently connected to the rest of the network).
* **Community Detection:** This bipartite graph can be projected into a **Product-to-Product** graph, where communities of products that are frequently co-purchased (potential bundles) can be found.
* **Recommendation Systems:** The graph structure is the bedrock of many **Collaborative Filtering** recommendation algorithms.

By using **`networkx`** to create this structured representation, your agent has gained the power to perform sophisticated **Topological Data Analysis** on its input data.

# Product Cooccurrence Graph
```
def build_product_cooccurrence_graph(
    transactions: List[Dict[str, Any]]
) -> nx.Graph:
    """
    Build product co-occurrence graph.
    
    Creates an undirected graph where:
    - Nodes are products
    - Edges connect products purchased by the same customer
    - Edge weights represent number of customers who bought both
    
    Args:
        transactions: List of transaction dictionaries
        
    Returns:
        NetworkX Graph object
    """
    G = nx.Graph()
    
    # Group transactions by customer
    customer_products = defaultdict(set)
    for txn in transactions:
        customer_id = txn.get("Customer_ID")
        product_id = txn.get("Product_ID")
        if customer_id and product_id:
            customer_products[customer_id].add(product_id)
    
    # Add product nodes
    all_products = set()
    for products in customer_products.values():
        all_products.update(products)
    
    for product_id in all_products:
        G.add_node(product_id, node_type="product")
    
    # Count co-occurrences
    cooccurrence = defaultdict(int)
    for customer_id, products in customer_products.items():
        products_list = list(products)
        # Create edges for all pairs of products this customer bought
        for i in range(len(products_list)):
            for j in range(i + 1, len(products_list)):
                p1, p2 = sorted([products_list[i], products_list[j]])
                cooccurrence[(p1, p2)] += 1
    
    # Add weighted edges
    for (p1, p2), weight in cooccurrence.items():
        G.add_edge(p1, p2, weight=weight, customers=weight)
    
    return G
```

This function, `build_product_cooccurrence_graph`, is analytically crucial. It moves the focus from individual customer actions to **structural relationships between products**, which is the prerequisite for discovering product bundles and cross-selling opportunities.

This graph is a **Product-only Projection** of your data, transforming the *bipartite* relationship into a *monopartite* (single-set) relationship.

***

## 🧠 Core Analytical Concept: Co-occurrence and Projection

### 1. Grouping Transactions (`customer_products = defaultdict(set)`)
* **Action:** This is the most important preparatory step. The code first aggregates all products bought by a single customer into a unique set.
* **Focus:** This ensures that if Customer C1 bought Product P01 yesterday and P02 today, the agent treats them as a single group of products purchased by that customer (`{P01, P02}`). The analysis is focused on **who bought what**, not *when* they bought it (though the `Transaction_Date` could be used for sequential analysis later).

### 2. Identifying and Counting Pairs (`cooccurrence = defaultdict(int)`)
* **Action:** The nested loops iterate over the unique products purchased by *each* customer and generate every possible unique pair of products (`(p1, p2)`).
* **Focus:** The use of `sorted([products_list[i], products_list[j]])` is a simple but critical programming pattern. It guarantees that the pair is always counted the same way (e.g., always `(A, B)` and never `(B, A)`), ensuring accurate weight summation.
* **The Power:** The final count, stored in `cooccurrence`, is the **Support Metric** (the number of times two items appeared together) for Association Rule Mining.

### 3. Graph Structure and Edge Weight
* **Nodes:** Every **unique product ID** becomes a node.
* **Edges:** An edge exists between two products if **at least one customer bought both**.
* **Weight:** The edge weight is the **number of customers** who purchased that specific pair. This weight is the direct measure of their co-dependence.

***

## ✨ Differentiation: Discovering Product Bundles

This function is directly responsible for powering the **"product\_bundling"** and **"association\_patterns"** goals in your orchestrator, making it vastly more powerful than a general agent.

* **Quantitative Bundling:** The resulting graph allows the `pattern_mining_agent` to immediately see which products are most strongly linked. Products with **high edge weights** are prime candidates for mandatory or recommended bundles.
* **Hidden Synergies:** This graph reveals subtle relationships. For example, if Products A and B are never advertised together, but 500 customers purchased both, that co-occurrence count (the edge weight) reveals a **hidden product synergy** that the business can exploit.
* **Visual Analysis:** The graph can be visualized , where the thickness of the line connecting two products immediately tells the story of their relationship strength.

By creating this structured graph, your agent moves from reporting past sales figures to **predicting future sales potential** based on structural relationships.

# Customer Similarity Graph

```
def build_customer_similarity_graph(
    transactions: List[Dict[str, Any]],
    min_shared_products: int = 2
) -> nx.Graph:
    """
    Build customer similarity graph based on shared products.
    
    Creates an undirected graph where:
    - Nodes are customers
    - Edges connect customers who share products
    - Edge weights represent number of shared products
    
    Args:
        transactions: List of transaction dictionaries
        min_shared_products: Minimum shared products to create edge
        
    Returns:
        NetworkX Graph object
    """
    G = nx.Graph()
    
    # Group products by customer
    customer_products = defaultdict(set)
    for txn in transactions:
        customer_id = txn.get("Customer_ID")
        product_id = txn.get("Product_ID")
        if customer_id and product_id:
            customer_products[customer_id].add(product_id)
    
    # Add customer nodes
    for customer_id in customer_products.keys():
        G.add_node(customer_id, node_type="customer")
    
    # Calculate similarity (shared products)
    customer_list = list(customer_products.keys())
    for i in range(len(customer_list)):
        for j in range(i + 1, len(customer_list)):
            c1, c2 = customer_list[i], customer_list[j]
            shared = customer_products[c1] & customer_products[c2]
            if len(shared) >= min_shared_products:
                G.add_edge(c1, c2, weight=len(shared), shared_products=list(shared))
    
    return G
```

This function is the third key graph utility, and it is specifically engineered to find **structurally similar customers**, which is the direct input needed for the **Customer Segmentation** focus area.

It is conceptually the **Customer-only Projection** of your data.

***

## 🧠 Core Analytical Concept: Behavioral Similarity

### 1. The Foundation: Shared Products
The primary goal is to redefine the customer base based on **shared purchasing behavior**. A simple customer list provides demographics; this function provides **predictive behavioral data**.

* **Step 1: Grouping (`customer_products = defaultdict(set)`):** Just as in the previous graph, the first step is to distill the transactions down to the **unique set of products** each customer owns. This product set becomes the customer's "behavioral fingerprint."
* **Step 2: Calculating Similarity via Set Intersection:**
    * The line `shared = customer_products[c1] & customer_products[c2]` is the analytical core. The **set intersection operator ($\&$)** is used to instantly find all products that both Customer 1 ($\text{c1}$) and Customer 2 ($\text{c2}$) have bought.
    * The **Edge Weight** is then set to the size of this shared set (`len(shared)`), making it a direct, quantifiable measure of similarity.

### 2. Filtering for Meaningful Connections
* **The Threshold:** The argument `min_shared_products: int = 2` and the condition `if len(shared) >= min_shared_products:` are vital.
* **Focus:** This acts as a **noise filter**. If two customers only share one obscure, cheap product, they aren't truly similar. By setting a minimum threshold, the resulting graph is sparse but **analytically rich**, containing only the strong, meaningful connections necessary for reliable segmentation.

### 3. Graph Structure and Output
* **Nodes:** Every **unique Customer ID** becomes a node, added with the `node_type="customer"` attribute.
* **Edges:** An edge connects any two customers who meet the `min_shared_products` threshold.
* **Weight:** The weight is the **number of shared products**, making the graph ready for clustering algorithms.

***

## ✨ Differentiation: Enabling Community Detection

This function makes your agent vastly more powerful by providing the necessary input for **Community Detection** algorithms, a key technique in modern data science.

* **Behavioral Segmentation:** The graph is the direct input for the **`clustering_agent`** (Step 3 in the plan). By applying graph-based clustering algorithms to this network, the agent can automatically identify **true behavioral communities** (segments) that are strongly inter-connected by their shared product choices.
* **Strategic Opportunity:** Analyzing this graph allows the agent to find customers on the *periphery* of a strong community. These are customers who are **very similar** to the core group (high edge weight) but who are currently *missing* one or two of the core products. This is a clear, actionable opportunity for **targeted marketing** and aligns perfectly with the goal of discovering **untapped market demand (ghost demand)**.

# Create Derived Features

```
def create_derived_features(
    customers: List[Dict[str, Any]],
    transactions: List[Dict[str, Any]],
    products: List[Dict[str, Any]]
) -> Dict[str, Any]:
    """
    Create derived features for analysis.
    
    Calculates:
    - Customer engagement scores (transaction frequency, product diversity)
    - Product popularity scores (usage frequency, customer count)
    - Usage intensity tiers (high/medium/low)
    
    Args:
        customers: List of customer dictionaries
        transactions: List of transaction dictionaries
        products: List of product dictionaries
        
    Returns:
        Dictionary with derived features
    """
    # Customer engagement metrics
    customer_txn_count = defaultdict(int)
    customer_products = defaultdict(set)
    customer_total_usage = defaultdict(float)
    
    for txn in transactions:
        customer_id = txn.get("Customer_ID")
        product_id = txn.get("Product_ID")
        usage = txn.get("Usage_Metric", 0)
        
        if customer_id:
            customer_txn_count[customer_id] += 1
            customer_total_usage[customer_id] += usage
            if product_id:
                customer_products[customer_id].add(product_id)
    
    # Calculate customer engagement scores
    customer_engagement = {}
    for customer in customers:
        customer_id = customer.get("Customer_ID")
        if customer_id:
            txn_count = customer_txn_count.get(customer_id, 0)
            product_diversity = len(customer_products.get(customer_id, set()))
            avg_usage = customer_total_usage.get(customer_id, 0) / max(txn_count, 1)
            
            # Simple engagement score (0-1 scale)
            engagement_score = min(1.0, (txn_count * 0.3 + product_diversity * 0.4 + (avg_usage / 100) * 0.3))
            
            customer_engagement[customer_id] = {
                "transaction_count": txn_count,
                "product_diversity": product_diversity,
                "average_usage": avg_usage,
                "engagement_score": engagement_score,
                "engagement_tier": "high" if engagement_score > 0.7 else "medium" if engagement_score > 0.4 else "low"
            }
    
    # Product popularity metrics
    product_txn_count = defaultdict(int)
    product_customers = defaultdict(set)
    product_total_usage = defaultdict(float)
    
    for txn in transactions:
        product_id = txn.get("Product_ID")
        customer_id = txn.get("Customer_ID")
        usage = txn.get("Usage_Metric", 0)
        
        if product_id:
            product_txn_count[product_id] += 1
            product_total_usage[product_id] += usage
            if customer_id:
                product_customers[product_id].add(customer_id)
    
    # Calculate product popularity scores
    product_popularity = {}
    for product in products:
        product_id = product.get("Product_ID")
        if product_id:
            txn_count = product_txn_count.get(product_id, 0)
            customer_count = len(product_customers.get(product_id, set()))
            avg_usage = product_total_usage.get(product_id, 0) / max(txn_count, 1)
            
            # Simple popularity score (0-1 scale)
            popularity_score = min(1.0, (txn_count * 0.4 + customer_count * 0.4 + (avg_usage / 100) * 0.2))
            
            product_popularity[product_id] = {
                "transaction_count": txn_count,
                "customer_count": customer_count,
                "average_usage": avg_usage,
                "popularity_score": popularity_score,
                "popularity_tier": "high" if popularity_score > 0.7 else "medium" if popularity_score > 0.4 else "low"
            }
    
    return {
        "customer_engagement": customer_engagement,
        "product_popularity": product_popularity
    }

```

This function, `create_derived_features`, is the **Feature Engineering** powerhouse of your agent, representing a crucial step that transforms raw numbers into **actionable, business-meaningful metrics**.

This is the code that **embeds business intelligence** directly into the data, making the downstream analysis infinitely more powerful.

***

## 🧠 Core Analytical Concept: Composite Scoring and Tiering

The function's architecture is a systematic application of Feature Engineering to create **composite scores** for two distinct entities: customers and products.

### 1. Customer Engagement Metrics

The agent defines **Customer Engagement** not by one metric, but by three weighted components:

| Component | Calculation | Business Weight (Score $\times$ Factor) | Strategic Purpose |
| :--- | :--- | :--- | :--- |
| **Transaction Count** | Total purchases. | $\times 0.3$ | Measures **Frequency** of interaction. |
| **Product Diversity** | Number of unique products purchased. | $\times 0.4$ | Measures **Breadth** and **Stickiness** (highest weight). |
| **Average Usage** | Total usage divided by transaction count. | $\times 0.3$ | Measures **Depth** of engagement per transaction. |

* **Composite Score:** The final `engagement_score` is a weighted average of these three factors, scaled to a 0-1 range. This score acts as a single, powerful input for the `clustering_agent`.
* **Tiering:** The `engagement_tier` (e.g., "high" if score $\text{>} 0.7$) converts this continuous number into a **categorical feature**. This is an excellent technique for simplifying the data for visualization and for the final **Synthesis Agent** (the LLM) to reason about.

### 2. Product Popularity Metrics

The agent applies the same logic to the product catalog:

* **Composite Score:** The `popularity_score` is calculated based on **Transaction Count** ($\times 0.4$), **Customer Count** ($\times 0.4$), and **Average Usage** ($\times 0.2$).
* **Strategic Encoding:** By giving high weight to both frequency ($\text{Txn Count}$) and reach ($\text{Customer Count}$), the score reflects a balanced view of market adoption. Products with high popularity scores are often core products, while low-scoring products might represent potential market gaps or failures.

***

## ✨ Differentiation: Bridging Data Science and Strategy

This function is the **strategic core** that elevates your orchestrator beyond simple data analysis

[Image of Feature Engineering Process]
.

* **Turning Data into Strategy:** Instead of passing the Synthesis Agent 10 raw columns, you pass it two highly-compressed, high-value metrics: `engagement_score` and `popularity_score`, along with clear `tiers`. This makes the final decision-making step more accurate and efficient.
* **Enabling Segmentation:** The `clustering_agent` will use these scores to create meaningful customer segments (e.g., "High Engagement/Low Diversity Customers" or "Low Engagement/High Diversity Customers"). These segments directly enable the discovery of **untapped market opportunities** (e.g., cross-selling a second product to a high-diversity customer who only bought one product).
* **Directly Addressing the Goal:** By explicitly calculating metrics for **Customer Segmentation** and features for **Strategic Opportunities**, this function is perfectly aligned with the overall goal defined in the `goal_node`.

# Tests for data preprocessing utilities

In [ ]:
"""Tests for data preprocessing utilities"""

import pytest
import networkx as nx
from tools.data_preprocessing import (
    parse_feature_set,
    normalize_usage_metrics,
    build_customer_product_graph,
    build_product_cooccurrence_graph,
    build_customer_similarity_graph,
    create_derived_features,
    preprocess_products
)


def test_parse_feature_set_single():
    """Test parsing single feature"""
    result = parse_feature_set("A")
    assert result == ["A"]


def test_parse_feature_set_multiple():
    """Test parsing multiple features"""
    result = parse_feature_set("A, B, C")
    assert result == ["A", "B", "C"]


def test_parse_feature_set_sorted():
    """Test features are sorted"""
    result = parse_feature_set("C, A, B")
    assert result == ["A", "B", "C"]


def test_parse_feature_set_with_spaces():
    """Test parsing with extra spaces"""
    result = parse_feature_set(" A , B , C ")
    assert result == ["A", "B", "C"]


def test_parse_feature_set_empty():
    """Test parsing empty string"""
    result = parse_feature_set("")
    assert result == []


def test_parse_feature_set_single_letter():
    """Test parsing single letter feature"""
    result = parse_feature_set("D")
    assert result == ["D"]


def test_normalize_usage_metrics():
    """Test usage metric normalization"""
    transactions = [
        {"Transaction_ID": "T1", "Usage_Metric": 10.0},
        {"Transaction_ID": "T2", "Usage_Metric": 50.0},
        {"Transaction_ID": "T3", "Usage_Metric": 90.0}
    ]

    result = normalize_usage_metrics(transactions)

    assert len(result) == 3
    assert "normalized_usage" in result[0]
    assert result[0]["normalized_usage"] == 0.0  # Min value
    assert result[2]["normalized_usage"] == 1.0  # Max value
    assert 0.0 <= result[1]["normalized_usage"] <= 1.0


def test_normalize_usage_metrics_empty():
    """Test normalization with empty list"""
    result = normalize_usage_metrics([])
    assert result == []


def test_build_customer_product_graph():
    """Test building customer-product bipartite graph"""
    customers = [
        {"Customer_ID": "C001", "Age_Group": "35-44"},
        {"Customer_ID": "C002", "Age_Group": "45-54"}
    ]
    products = [
        {"Product_ID": "P01", "Product_Type": "Hardware"},
        {"Product_ID": "P02", "Product_Type": "Software"}
    ]
    transactions = [
        {"Customer_ID": "C001", "Product_ID": "P01"},
        {"Customer_ID": "C001", "Product_ID": "P02"},
        {"Customer_ID": "C002", "Product_ID": "P01"}
    ]

    G = build_customer_product_graph(customers, transactions, products)

    assert isinstance(G, nx.Graph)
    assert "C001" in G.nodes
    assert "C002" in G.nodes
    assert "P01" in G.nodes
    assert "P02" in G.nodes
    assert G.has_edge("C001", "P01")
    assert G.has_edge("C001", "P02")
    assert G.has_edge("C002", "P01")
    assert G["C001"]["P01"]["weight"] == 1


def test_build_product_cooccurrence_graph():
    """Test building product co-occurrence graph"""
    transactions = [
        {"Customer_ID": "C001", "Product_ID": "P01"},
        {"Customer_ID": "C001", "Product_ID": "P02"},
        {"Customer_ID": "C002", "Product_ID": "P01"},
        {"Customer_ID": "C002", "Product_ID": "P02"},
        {"Customer_ID": "C003", "Product_ID": "P02"},
        {"Customer_ID": "C003", "Product_ID": "P03"}
    ]

    G = build_product_cooccurrence_graph(transactions)

    assert isinstance(G, nx.Graph)
    assert "P01" in G.nodes
    assert "P02" in G.nodes
    assert "P03" in G.nodes
    assert G.has_edge("P01", "P02")
    assert G["P01"]["P02"]["weight"] == 2  # C001 and C002 both have P01 and P02
    assert G.has_edge("P02", "P03")
    assert G["P02"]["P03"]["weight"] == 1  # Only C003 has both


def test_build_customer_similarity_graph():
    """Test building customer similarity graph"""
    transactions = [
        {"Customer_ID": "C001", "Product_ID": "P01"},
        {"Customer_ID": "C001", "Product_ID": "P02"},
        {"Customer_ID": "C002", "Product_ID": "P01"},
        {"Customer_ID": "C002", "Product_ID": "P02"},
        {"Customer_ID": "C003", "Product_ID": "P03"}
    ]

    G = build_customer_similarity_graph(transactions, min_shared_products=2)

    assert isinstance(G, nx.Graph)
    assert "C001" in G.nodes
    assert "C002" in G.nodes
    assert "C003" in G.nodes
    assert G.has_edge("C001", "C002")  # Share P01 and P02
    assert G["C001"]["C002"]["weight"] == 2
    assert not G.has_edge("C001", "C003")  # No shared products


def test_create_derived_features():
    """Test creating derived features"""
    customers = [
        {"Customer_ID": "C001"},
        {"Customer_ID": "C002"}
    ]
    products = [
        {"Product_ID": "P01"},
        {"Product_ID": "P02"}
    ]
    transactions = [
        {"Customer_ID": "C001", "Product_ID": "P01", "Usage_Metric": 50.0},
        {"Customer_ID": "C001", "Product_ID": "P02", "Usage_Metric": 60.0},
        {"Customer_ID": "C002", "Product_ID": "P01", "Usage_Metric": 70.0}
    ]

    result = create_derived_features(customers, transactions, products)

    assert "customer_engagement" in result
    assert "product_popularity" in result
    assert "C001" in result["customer_engagement"]
    assert "P01" in result["product_popularity"]
    assert "engagement_score" in result["customer_engagement"]["C001"]
    assert "popularity_score" in result["product_popularity"]["P01"]


def test_preprocess_products():
    """Test preprocessing products with Feature_Set parsing"""
    products = [
        {"Product_ID": "P01", "Feature_Set": "A, B"},
        {"Product_ID": "P02", "Feature_Set": "C"}
    ]

    result = preprocess_products(products)

    assert len(result) == 2
    assert "feature_list" in result[0]
    assert result[0]["feature_list"] == ["A", "B"]
    assert result[1]["feature_list"] == ["C"]



In [ ]:
(.venv) micahshull@Micahs-iMac LG_Cursor_035_Product-CustomerFitDiscoveryOrchestrator % python3 -m pytest tests/test_data_preprocessing.py -v

============================================================ test session starts ============================================================
platform darwin -- Python 3.13.7, pytest-9.0.1, pluggy-1.6.0 -- /Users/micahshull/Documents/AI_LangGraph/LG_Cursor_035_Product-CustomerFitDiscoveryOrchestrator/.venv/bin/python3
cachedir: .pytest_cache
rootdir: /Users/micahshull/Documents/AI_LangGraph/LG_Cursor_035_Product-CustomerFitDiscoveryOrchestrator
plugins: langsmith-0.4.53, anyio-4.12.0, asyncio-1.3.0, cov-7.0.0
asyncio: mode=Mode.STRICT, debug=False, asyncio_default_fixture_loop_scope=None, asyncio_default_test_loop_scope=function
collected 13 items

tests/test_data_preprocessing.py::test_parse_feature_set_single PASSED                                                                [  7%]
tests/test_data_preprocessing.py::test_parse_feature_set_multiple PASSED                                                              [ 15%]
tests/test_data_preprocessing.py::test_parse_feature_set_sorted PASSED                                                                [ 23%]
tests/test_data_preprocessing.py::test_parse_feature_set_with_spaces PASSED                                                           [ 30%]
tests/test_data_preprocessing.py::test_parse_feature_set_empty PASSED                                                                 [ 38%]
tests/test_data_preprocessing.py::test_parse_feature_set_single_letter PASSED                                                         [ 46%]
tests/test_data_preprocessing.py::test_normalize_usage_metrics PASSED                                                                 [ 53%]
tests/test_data_preprocessing.py::test_normalize_usage_metrics_empty PASSED                                                           [ 61%]
tests/test_data_preprocessing.py::test_build_customer_product_graph PASSED                                                            [ 69%]
tests/test_data_preprocessing.py::test_build_product_cooccurrence_graph PASSED                                                        [ 76%]
tests/test_data_preprocessing.py::test_build_customer_similarity_graph PASSED                                                         [ 84%]
tests/test_data_preprocessing.py::test_create_derived_features PASSED                                                                 [ 92%]
tests/test_data_preprocessing.py::test_preprocess_products PASSED                                                                     [100%]

============================================================ 13 passed in 0.13s =============================================================


In [ ]:
(.venv) micahshull@Micahs-iMac LG_Cursor_035_Product-CustomerFitDiscoveryOrchestrator % python3 -m pytest tests/test_nodes_phase3.py -v
============================================================ test session starts ============================================================
platform darwin -- Python 3.13.7, pytest-9.0.1, pluggy-1.6.0 -- /Users/micahshull/Documents/AI_LangGraph/LG_Cursor_035_Product-CustomerFitDiscoveryOrchestrator/.venv/bin/python3
cachedir: .pytest_cache
rootdir: /Users/micahshull/Documents/AI_LangGraph/LG_Cursor_035_Product-CustomerFitDiscoveryOrchestrator
plugins: langsmith-0.4.53, anyio-4.12.0, asyncio-1.3.0, cov-7.0.0
asyncio: mode=Mode.STRICT, debug=False, asyncio_default_fixture_loop_scope=None, asyncio_default_test_loop_scope=function
collected 10 items

tests/test_nodes_phase3.py::test_data_preprocessing_node_basic PASSED                                                                 [ 10%]
tests/test_nodes_phase3.py::test_data_preprocessing_node_structure PASSED                                                             [ 20%]
tests/test_nodes_phase3.py::test_data_preprocessing_node_parses_feature_sets PASSED                                                   [ 30%]
tests/test_nodes_phase3.py::test_data_preprocessing_node_normalizes_usage PASSED                                                      [ 40%]
tests/test_nodes_phase3.py::test_data_preprocessing_node_builds_graphs PASSED                                                         [ 50%]
tests/test_nodes_phase3.py::test_data_preprocessing_node_creates_derived_features PASSED                                              [ 60%]
tests/test_nodes_phase3.py::test_data_preprocessing_node_data_quality_report PASSED                                                   [ 70%]
tests/test_nodes_phase3.py::test_data_preprocessing_node_requires_raw_data PASSED                                                     [ 80%]
tests/test_nodes_phase3.py::test_data_preprocessing_node_preserves_errors PASSED                                                      [ 90%]
tests/test_nodes_phase3.py::test_data_preprocessing_node_full_workflow PASSED                                                         [100%]

============================================================ 10 passed in 0.47s =============================================================




## Phase 3 complete — all tests passing

### Summary

**Phase 1: Foundation**
- Goal Node ✓
- Planning Node ✓
- 7/7 tests passing

**Phase 2: Data Loading**
- Data Loading Utilities (3 functions) ✓
- Data Ingestion Node ✓
- 12 utility + 9 node = 21/21 tests passing

**Phase 3: Data Preprocessing**
- Preprocessing Utilities (7 functions) ✓
- Data Preprocessing Node ✓
- 13 utility + 10 node = 23/23 tests passing

**Total progress:** 51/51 tests passing across all phases

---

### What we've built

1. Complete data pipeline:
   - Goal → Planning → Data Ingestion → Data Preprocessing
   - All nodes tested independently and in workflow

2. Graph structures:
   - Customer-Product bipartite graph
   - Product co-occurrence graph
   - Customer similarity graph

3. Derived features:
   - Customer engagement scores
   - Product popularity scores
   - Normalized usage metrics

---

## Next: Phase 4 — Clustering Agent

This phase will:
1. Build clustering utilities (K-means for customers and products)
2. Analyze cluster characteristics
3. Identify underserved segments
4. Build clustering agent node

